# Lab 05-2: Simple Neural Network
## Exercise: Predicting Iris Species

### Prepare IRIS Dataset

In [ ]:
import numpy as np
import pandas as pd

from sklearn.datasets import load_iris

iris = load_iris()

# iris.data contains four column
#   sepal length (cm) / sepal width (cm) / petal length (cm) / petal width (cm)
# iris.target contains one column
#   species of (0,1,2) = (setosa, versicolor, virginica)
iris_df = pd.DataFrame(data= iris.data, columns= iris.feature_names)
iris_tf = pd.DataFrame(data= iris.target, columns= ['species'])

vX = iris_df.copy()
vY = iris_tf['species']

# Chnage dataset from pandas to numpy
vX = vX.to_numpy()
vY = vY.to_numpy()

### Presenting Dataset Samples

In [ ]:
iris_df = pd.concat([iris_df, iris_tf], axis= 1)
iris_df.describe()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),species
count,150.000000,150.000000,150.000000,150.000000,150.000000
mean,5.843333,3.057333,3.758000,1.199333,1.000000
std,0.828066,0.435866,1.765298,0.762238,0.819232
min,4.300000,2.000000,1.000000,0.100000,0.000000
25%,5.100000,2.800000,1.600000,0.300000,0.000000
50%,5.800000,3.000000,4.350000,1.300000,1.000000
75%,6.400000,3.300000,5.100000,1.800000,2.000000
max,7.900000,4.400000,6.900000,2.500000,2.000000


In [ ]:
print(vY)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2]


Splitting Data for Training and Testing, then Preparing one-hot labeled ground truth

In [ ]:
# We can use train_test_split from sklearn
from sklearn.model_selection import train_test_split

# Splitting dataframe into train & test
X_train, X_test, y_train_num, y_test = train_test_split(vX, vY, test_size= 0.20, random_state= 101)

# species are 0 for setosa, 1 for versicolor, and 2 for virginica
# make one-hot training sequence from numerical category sequence
n_train = y_train_num.shape[0]
y_train = np.zeros((n_train, 3))
for i in range(n_train):
    y_train[i, y_train_num[i]] = 1

Define Utility Functions

In [ ]:
def sigmoid(x):
    # Numerically stable with large exponentials
    x = np.where(x < 0, np.exp(x)/(1 + np.exp(x)), 1/(1 + np.exp(-x)))
    return x

def softmax(x):
    # Numerically stable with large exponentials
    x = x - np.max(x, axis=-1, keepdims=True)
    x = np.exp(x)
    xs = np.sum(x, axis=-1, keepdims=True)
    return x / xs

### Simple Neural Network

Define Model Class for Linear Preiction (i.e., no activation)<br>

$$z^{[l]} = W^{[l]}x^{[l]} + b^{[l]}, \;\text{where}\;\; x^{[l]}=a^{[l-1]}$$

$${\partial J \over \partial W^{[l]}} = {\partial J \over \partial z^{[l]}} \cdot a^{[l-1]}, \qquad
{\partial J \over \partial b} = {\partial J \over \partial z^{[l]}}$$

**Note:** $\partial J \over \partial W^{[l]}$ should have the same shape as $W^{[l]}$. You must transpose dimensions appropriately.

In [ ]:
class myNeuralLayer:
    def __init__(self, n_out, n_in):
        self.wegt = np.zeros((n_out, n_in))
        self.bias = np.zeros((n_out))

    def forward(self, x):                               # (b, i)
        ### START CODE HERE ###

        x_lin = x @ self.wegt.T + self.bias             # Linear Prediction
        
        ### END CODE HERE ###
        return x_lin

    def backward(self, x, a_in):  # x = dJ/dz (b, c), a_in = a(l-1)
        ### START CODE HERE ###
        
        dw = x.T @ a_in / x.shape[0]               # Gradients for weights
        db = np.mean(x, axis=0)               # Gradients for biases
        wdJdz = x @ self.wegt            # Propagation for lower layer
        
        ### END CODE HERE ###
        return dw, db, wdJdz


Test Model Class

In [ ]:
nn = myNeuralLayer(2, 3)
nn.wegt = np.arange(6).reshape(2, 3)

a = np.array([[1,2,3], [3,4,5]])
b = np.array([[1,2], [3,4]])
c = nn.forward(a)
d, e, f = nn.backward(b, a)
print('c = [', c[0], c[1], ']\nd = [', d[0], d[1], ']\ne = ', e, '\nf = [', f[0], f[1], ']')

c = [ [ 8. 26.] [14. 50.] ]
d = [ [5. 7. 9.] [ 7. 10. 13.] ]
e =  [2. 3.] 
f = [ [ 6  9 12] [12 19 26] ]


**Expected Outputs**
```
c = [ [ 8. 26.] [14. 50.] ]
d = [ [5. 7. 9.] [ 7. 10. 13.] ]
e =  [2. 3.] 
f = [ [ 6  9 12] [12 19 26] ]
```

Create a NN model and check the matrix dimensions

In [ ]:
n_inputs  = 4
n_hiddens = 5
n_classes = 3

l1 = myNeuralLayer(n_hiddens, n_inputs)
l2 = myNeuralLayer(n_classes, n_hiddens)

print(X_train.shape, y_train.shape)
print(l1.wegt.shape, l1.bias.shape)
print(l2.wegt.shape, l2.bias.shape)

(120, 4) (120, 3)
(5, 4) (5,)
(3, 5) (3,)


Weight Initialization<br>

There are many delicate training issues related to weight initialization. 
- Check what happen if weights are initialized to...
- np.zeros / np.ones / np.random.rand / np.random.randn

In [ ]:
# Weight Initialization

### START CODE HERE ###

# change codes for each initialization function
l1.wegt = np.random.randn(n_hiddens, n_inputs)
l2.wegt = np.random.randn(n_classes, n_hiddens)

### END CODE HERE ###

Define Backpropagation of Activation Functions

Define Model Class for Linear Preiction (i.e., no activation)<br>

$$z^{[l]} = W^{[l]}x^{[l]} + b^{[l]}, \qquad
J = -{1 \over n} \sum_{i=1}^{n} \left(y (Wx^{(i)} + b) - \log(1+e^{Wx^{(i)} + b}) \right)$$

$${\partial J \over \partial W} = {1 \over n} \sum_{i=1}^{n} \left(\left(y - h(x^{(i)})\right) \cdot x_j^{(i)}\right), \qquad
{\partial J \over \partial b} = {1 \over n} \sum_{i=1}^{n} \left(y - h(x^{(i)})\right)$$

Sigmoid is used in the middle of networks, while softmax is used at the end of networks.<br>
Therefore the `dJdz_softmax` is a combined backward path from cross-entropy cost to softmax.

$${\partial J \over \partial z^{[l]}} = \left( \left( {\partial J \over \partial z^{[l+1]}} \right)^T \cdot W^{[l+1]} \right)^T \odot \sigma ' (z^{[l]})$$

$${\partial J \over \partial z^{[L]}} = \hat{y} - y$$

In [ ]:
def dJdz_sigmoid(wdJdz_upper, az):
    ### START CODE HERE ###

    dJdz = wdJdz_upper * az * (1 - az)            # backpropagation through activation function
    
    ### END CODE HERE ###
    return dJdz

def dJdz_softmax(y_hat, y):
    ### START CODE HERE ###
    
    dJdz = y_hat - y            # backpropagation form cross-entropy to softmax
    
    ### END CODE HERE ###
    return dJdz

Training Simple Neural Network Model (2 layer model)

In [ ]:
# define learning rate alpha, and number of epochs
alpha = 0.1
n_epochs = 5000

for epoch in range(n_epochs):
    ### START CODE HERE ###

    # Forward Stages
    a_1 = sigmoid(l1.forward(X_train))                    # first stage forward; linear and activation fn
    a_2 = softmax(l2.forward(a_1))                    # second stage forward; linear and activation fn
    
    # Backward Stages
    dJdz_2 = dJdz_softmax(a_2, y_train)                 # Find dJ/dz for the 2nd stage backward
    dw_2, db_2, wdJdz_2 = l2.backward(dJdz_2, a_1)    # go through backward for linear transformation
    dJdz_1 = dJdz_sigmoid(wdJdz_2, a_1)                # FInd dJ/dz for the 1st stage backward
    dw_1, db_1, _ = l1.backward(dJdz_1, X_train)          # go through backward for linear transformation

    # Update weights and biases
    l2.wegt = l2.wegt - alpha * dw_2
    l2.bias = l2.bias - alpha * db_2
    l1.wegt = l1.wegt - alpha * dw_1
    l1.bias = l1.bias - alpha * db_1

    ### END CODE HERE ###

    # Print loss values
    if ((epoch+1)%500==0):
        ### START CODE HERE ###

        y_hidd = sigmoid(X_train @ l1.wegt.T + l1.bias)             # first stage forward
        y_prob = softmax(y_hidd @ l2.wegt.T + l2.bias)             # second stage forward
        loss_J = -np.sum(y_train * np.log(y_prob))             # Calculate loss
        ### END CODE HERE ###
        print('Epoch: %4d,  loss: %10.8f' % (epoch+1, loss_J))

(120, 3) (120, 3)
Epoch:  500,  loss: 7.76362263
(120, 3) (120, 3)
Epoch: 1000,  loss: 7.58821464
(120, 3) (120, 3)
Epoch: 1500,  loss: 7.43877681
(120, 3) (120, 3)
Epoch: 2000,  loss: 7.30910539
(120, 3) (120, 3)
Epoch: 2500,  loss: 7.19485540
(120, 3) (120, 3)
Epoch: 3000,  loss: 7.09289014
(120, 3) (120, 3)
Epoch: 3500,  loss: 7.00088643
(120, 3) (120, 3)
Epoch: 4000,  loss: 6.91708617
(120, 3) (120, 3)
Epoch: 4500,  loss: 6.84013471
(120, 3) (120, 3)
Epoch: 5000,  loss: 6.76897261


Evaluate Model Performance

In [ ]:
def my_predict(l1, l2, X_test):
    ### START CODE HERE ###

    y_hidd = sigmoid(X_test @ l1.wegt.T + l1.bias)               # first stage forward
    y_prob = softmax(y_hidd @ l2.wegt.T + l2.bias)               # second stage forward
    y_pred = np.argmax(y_prob, axis=1)               # make prediction

    ### END CODE HERE ###
    return y_pred

from sklearn.metrics import accuracy_score

y_pred = my_predict(l1, l2, X_test)

print(y_pred)
print(y_test)

accuracy_score(y_pred, y_test)

[0 0 0 2 1 2 1 1 2 0 2 0 0 2 2 1 1 1 0 2 1 0 1 1 1 1 1 2 0 0]
[0 0 0 2 1 2 1 1 2 0 2 0 0 2 2 1 1 1 0 2 1 0 1 1 1 1 1 2 0 0]


1.0

Neural Network from scikit-learn

In [ ]:
from sklearn.neural_network import MLPClassifier

mlp = MLPClassifier(hidden_layer_sizes=(5,), activation='logistic', solver='sgd', \
                    alpha=0.01, learning_rate_init=0.1, max_iter=500)

# Training/Fitting the Model
mlp.fit(X_train, y_train_num)

# Making Predictions
pred = mlp.predict(X_test)
accuracy_score(pred, y_test)

1.0

### Test Model with a random sample


In [ ]:
idx = np.random.randint(X_test.shape[0])
test_in = np.expand_dims(X_test[idx], axis=0)

species = ['setosa', 'versicolor', 'virginica']

y_pred = my_predict(l1, l2, test_in)
s_pred = mlp.predict(test_in)

print('My Prediction for Iris Species:', species[y_pred[0]])
print('SK Prediction for Iris Species:', species[s_pred[0]])
print('True Iris Species is:', species[y_test[idx]])

My Prediction for Iris Species: setosa
SK Prediction for Iris Species: setosa
True Iris Species is: setosa
